# Retirement Tool


# Retirement Planning Tool

Part 1: Data inputs will be generated to determine the years in retirement, size of current portfolio, and historical growth rates of indices.

Part 2: Users will forecast the performance of their portfolio at the age they wish to retire until they turn 80. Historical price data for 3 years will be used to generate Monte Carlo simulations to compute total savings (mean) for the first year of retirement. We are assuming that it will cost 150,000 per year to retire in NYC or 2.25 million to retire at 65.

Part 3: This total will be exported to the Cost of Living Calculator to determine the list of cities where the user can feasibly retire.

In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import sys
from tkinter import Y
import fire
import questionary
from pathlib import Path

%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

## Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv("SAMPLE.env")
Step 1: Questionary inputs and "For Loop" to generate list of qualifying cities.

In [ ]:
# Inputs

#age = input(int("What is your current age?")
#retirement_age = input(int("When would you like to retire?")
#age_of_death = 80
#retirement_years = age_of_death - retirement_age
#years_to_retirment = retirement_age - age
#savings = input(int("How much do you have in your savings account today?")
#cost_of_living = #listofcitieswithcostofliving
#portfolio_type = input(int("Would you like your portfolio to be conservative, conservatively moderate, or moderate?"))
#stock_portoflio_$amount =input(int("How much do you have in your stock account?")
#bond_portoflio_$amount =input(int("How much do you have in your bond account?")

def get_retire_plan():

    age = questionary.text("What is your current age").ask()
    retirement_age = questionary.text("By what age, would you prefer to retire?").ask()
    savings = questionary.text("How much do you have in liquid cash savings (USD)").ask()
    portfolio_type = questionary.text("Would you like your portfolio to be conservative[1], conservatively moderate[2], or moderate[3]? (Enter 1, 2, or 3)").ask()
    stock_portfolio_amount = questionary.text("What is the total cash value of stock holdings?").ask()
    bond_portfolio_amount = questionary.text("What is the total cash value of bond holdings?").ask()
        
    age = int(age)
    retirement_age = int(retirement_age)
    savings = float(savings)
    portfolio_type = int(portfolio_type)
    stock_portfolio_amount = float(stock_portfolio_amount)
    bond_portfolio_amount = float(bond_portfolio_amount)

    return age, retirement_age, savings, portfolio_type, stock_portfolio_amount, bond_portfolio_amount

age_of_death = int(80)
#retirement_years = int(age_of_death - retirement_age)
#years_to_retirement = int(retirement_age - age)
#total_stocks_bonds = float(stock_portfolio_amount + bond_portfolio_amount)

def run_app():
    """The main function for running the script."""

    # Get the applicant's information
    age, retirement_age, savings, portfolio_type, stock_portfolio_amount, bond_portfolio_amount = get_retire_plan()

if __name__ == "__main__":
    fire.Fire(run_app)

In [ ]:
tickers=["SPY", "AGG"]



#For loop to create a list of cities the individual can afford to retire in based on their savings and portfolio

#Assigning value in order to run code. Section to be added




In [ ]:
load_dotenv()

### Get data from Alpaca for MC Simulation of retirement portfolio.

In [ ]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

Step 1: Create a Python list named savings_data that has two elements. The first element contains the total value of the cryptocurrency wallet. The second element contains the total value of the stock and bond portions of the portfolio.

In [ ]:
# Consolidate financial assets data into a Python list
savings_data=[]

savings_data.append(savings)
savings_data.append(total_stocks_bonds)
    

# Review the Python list savings_data
savings_data  

Step 2: Use the savings_data list to create a Pandas DataFrame named savings_df, and then display this DataFrame. The function to create the DataFrame should take the following three parameters:
savings_data: Use the list that you just created.

columns: Set this parameter equal to a Python list with a single value called amount.

index: Set this parameter equal to a Python list with the values of crypto and stock/bond.

In [ ]:
# Create a Pandas DataFrame called savings_df 
index=["cash", "stock/bond"]
columns=["amount"]
savings_df = pd.DataFrame(savings_data , columns=columns, index=index)

# Display the savings_df DataFrame
savings_df

Part 2: Create a Financial Planner for Retirement

Step 1: Make an API call via the Alpaca SDK to get 3 years of historical closing prices for a traditional 60/40 portfolio split: 60% stocks (SPY) and 40% bonds (AGG).

In [ ]:
# Set start and end dates of 3 years back from your current date
# Alternatively, you can use an end date of 2020-08-07 and work 3 years back from that date 
start_date = pd.Timestamp("2019-04-22", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-04-22", tz="America/New_York").isoformat()

# Set number of rows to 1000 to retrieve the maximum amount of rows
limit_rows=10000

In [ ]:
# Use the Alpaca get_bars function to make the API call to get the 3 years worth of pricing data
# The tickers and timeframe parameters should have been set in Part 1 of this activity 
# The start and end dates should be updated with the information set above
# Remember to add the df property to the end of the call so the response is returned as a DataFrame
timeframe="1Day"

df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit=limit_rows
).df



# Reorganize the DataFrame
# Separate ticker data
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
AGG = df_portfolio[df_portfolio['symbol']=='AGG'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_portfolio = pd.concat([SPY,AGG],axis=1, keys=['SPY','AGG'])


# Display both the first and last five rows of the DataFrame
display(df_portfolio.head())
display(df_portfolio.tail())

Step 2: Run a Monte Carlo simulation of 500 samples and 30 years for the 60/40 portfolio, and then plot the results.

In [ ]:
# Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# The weights should be split 40% to FXNAX and 60% to SPY.
# Run 500 samples.

#years_to_retirment = retirement_age - age

def MonteCarlo():
    if portfolio_type == 'Moderate':
        MonteCarlo_output = MCSimulation(
            portfolio_data = df_portfolio,
            weights = [.63,.37],
            num_simulation = 500,
            num_trading_days = 252*years_to_retirement
)
        return MonteCarlo_output
        
    elif portfolio_type == "Conservative":
        MonteCarlo_output = MCSimulation(
            portfolio_data = df_portfolio,
            weights = [.44,.56],
            num_simulation = 500,
            num_trading_days = 252*years_to_retirement
)

        return MonteCarlo_output
        
    else:
        MonteCarlo_output == MCSimulation(
            portfolio_data = df_portfolio,
            weights = [.29,.71],
            num_simulation = 500,
            num_trading_days = 252*years_to_retirement
)

        return MonteCarlo_output

# Review the simulation input data

output = MonteCarlo()

output.portfolio_data.dropna()



In [ ]:
# Run the Monte Carlo simulation to forecast 30 years cumulative returns
output.calc_cumulative_return()

In [ ]:
# Visualize the 30-year Monte Carlo simulation by creating an
# overlay line plot
MC_sim_line_plot = output.plot_simulation()

In [ ]:
Step 3: Plot the probability distribution of the Monte Carlo simulation.

In [ ]:
# Visualize the probability distribution of the 30-year Monte Carlo simulation 
# by plotting a histogram
MC_sim_dist_plot = output.plot_distribution()

In [ ]:
Step 4: Generate the summary statistics for the Monte Carlo simulation.
# Generate summary statistics from the {years_to_retirement} Monte Carlo simulation results
# Save the results as a variable
MC_summary_statistics = output.summarize_cumulative_return()


# Review the {years_to_retirement} Monte Carlo summary statistics
print(MC_summary_statistics)

Analyze the Retirement Portfolio Forecasts
Using the current value of only the stock and bond portion of the member's portfolio and the summary statistics that you generated from the Monte Carlo simulation, answer the following question in your Jupyter notebook:

What are the lower and upper bounds for the expected value of the portfolio with a 95% confidence interval?

In [ ]:
# Print the current balance of the stock and bond portion of the members portfolio
print(f"The current balance of the stock and bond portion of the members' portfolio is: ${total_stocks_bonds}")
total_stocks_bonds

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
#ci_lower_thirty_cumulative_return = MC_summary_statistics[8] * float(total_stocks_bonds)
#ci_upper_thirty_cumulative_return = MC_summary_statistics[9] * float(total_stocks_bonds)
mean_cumulative_return = MC_summary_statistics[5] * float(total_stocks_bonds)


# Print the result of your calculations
print(f"The initial investment of ${total_stocks_bonds} in the portfolio"
    f" over the next {years_to_retirement} will end on average"
    f" ${mean_cumulative_return: .2f}.")

In [ ]:
Sum the total forecasted investment with the cash savings to arrive at the reitrement savings amount.

In [ ]:
savings_at_retirement = float(mean_cumulative_return + savings)
savings_at_retirement

In [ ]:
retirement_cities = []
                
costOfLivingByCity = [
    {
        "city": "Hamilton",
        "cost_of_living_index": 147.81,
        "cost_of_living_plus_rent_index": 119.78,
        "country": "Bermuda",
        "estimated_monthly_costs_with_rent": 3665.6,
        "estimated_monthly_costs_without_rent": 1855.53,
        "groceries_index": 154.4,
        "local_purchasing_power_index": 88.1,
        "rent_index": 88.78,
        "restaurant_price_index": 154.25
    },
    {
        "city": "New York",
        "cost_of_living_index": 100.0,
        "cost_of_living_plus_rent_index": 100.0,
        "country": "United States",
        "estimated_monthly_costs_with_rent": 3616.0200000000004,
        "estimated_monthly_costs_without_rent": 1364.22,
        "groceries_index": 100.0,
        "local_purchasing_power_index": 100.0,
        "rent_index": 100.0,
        "restaurant_price_index": 100.0
    },
    {
        "city": "Hong Kong",
        "cost_of_living_index": 79.62,
        "cost_of_living_plus_rent_index": 75.4,
        "country": "Hong Kong",
        "estimated_monthly_costs_with_rent": 2575.87,
        "estimated_monthly_costs_without_rent": 1083.08,
        "groceries_index": 82.6,
        "local_purchasing_power_index": 66.05,
        "rent_index": 70.74,
        "restaurant_price_index": 58.38
    },
    {
        "city": "Los Angeles",
        "cost_of_living_index": 78.53,
        "cost_of_living_plus_rent_index": 75.87,
        "country": "United States",
        "estimated_monthly_costs_with_rent": 2710.7200000000003,
        "estimated_monthly_costs_without_rent": 1052.74,
        "groceries_index": 74.39,
        "local_purchasing_power_index": 126.07,
        "rent_index": 72.93,
        "restaurant_price_index": 91.42
    },
    {
        "city": "Paris",
        "cost_of_living_index": 81.85,
        "cost_of_living_plus_rent_index": 65.33,
        "country": "France",
        "estimated_monthly_costs_with_rent": 2045.1799999999998,
        "estimated_monthly_costs_without_rent": 1064.56,
        "groceries_index": 80.94,
        "local_purchasing_power_index": 78.38,
        "rent_index": 47.07,
        "restaurant_price_index": 79.53
    },
    {
        "city": "Hamilton",
        "cost_of_living_index": 66.24,
        "cost_of_living_plus_rent_index": 52.01,
        "country": "Canada",
        "estimated_monthly_costs_with_rent": 1940.19,
        "estimated_monthly_costs_without_rent": 869.11,
        "groceries_index": 66.36,
        "local_purchasing_power_index": 85.47,
        "rent_index": 36.29,
        "restaurant_price_index": 64.04
    },
    {
        "city": "Milan",
        "cost_of_living_index": 74.46,
        "cost_of_living_plus_rent_index": 59.61,
        "country": "Italy",
        "estimated_monthly_costs_with_rent": 1891.1,
        "estimated_monthly_costs_without_rent": 968.81,
        "groceries_index": 62.21,
        "local_purchasing_power_index": 53.54,
        "rent_index": 43.19,
        "restaurant_price_index": 79.33
    },
    {
        "city": "Bucaramanga",
        "cost_of_living_index": 24.7,
        "cost_of_living_plus_rent_index": 15.71,
        "country": "Colombia",
        "estimated_monthly_costs_with_rent": 481.11,
        "estimated_monthly_costs_without_rent": 342.29,
        "groceries_index": 19.63,
        "local_purchasing_power_index": 29.77,
        "rent_index": 5.78,
        "restaurant_price_index": 17.74
    },
    {
        "city": "Madrid",
        "cost_of_living_index": 58.87,
        "cost_of_living_plus_rent_index": 45.63,
        "country": "Spain",
        "estimated_monthly_costs_with_rent": 1552.51,
        "estimated_monthly_costs_without_rent": 763.4,
        "groceries_index": 48.64,
        "local_purchasing_power_index": 73.53,
        "rent_index": 30.98,
        "restaurant_price_index": 61.83
    },
    {
        "city": "Delhi",
        "cost_of_living_index": 27.75,
        "cost_of_living_plus_rent_index": 18.3,
        "country": "India",
        "estimated_monthly_costs_with_rent": 502.83,
        "estimated_monthly_costs_without_rent": 369.77,
        "groceries_index": 27.03,
        "local_purchasing_power_index": 57.41,
        "rent_index": 7.86,
        "restaurant_price_index": 24.19
    }
]               
                
for city in costOfLivingByCity:
    if savings_at_retirement >= (city['estimated_monthly_costs_with_rent'] * (retirement_years * 12)):                
        retirement_cities.append((city)) 
print(retirement_cities)

